# What is in the channel files that lsdtopotools produces

Last updated by Simon M Mudd 25/04/2023

In this notebook we will use an example where you have collected some channel characteristics in the field and we want to know the drainage area of the points. This will include the simplest possible example where all we have is the location of the points. 

## Stuff we need to do if you are in colab (not required in the lsdtopotools pytools container)

**If you are in the `docker_lsdtt_pytools` docker container, you do not need to do any of this. 
The following is for executing this code in the google colab environment only.**

If you are in the docker container you can skip to the **Download some data** section. 

**If you are in the `docker_lsdtt_pytools` docker container, you do not need to do any of this. 
The following is for executing this code in the google colab environment only.**

If you are in the docker container you can skip to the **First get data** section. 

First we install `lsdviztools`. 
This will take around a minute.

In [ ]:
!pip install lsdviztools &> /dev/null

Now we need to install lsdtopotools. We do this using something called `mamba`. Note that this version of `mamba` works for python 3.9 (which is what google colab currently uses).

This step will take around 20 seconds. 

In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Mambaforge-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-Linux-x86_64.sh &> /dev/null
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX &> /dev/null

Alternatively we can do this with condacolab, but this broke in March 2023 so will take some time to be fixed

In [ ]:
#!pip install -q condacolab
#import condacolab
#condacolab.install()

Now use mamba to install `lsdtopotools`. 
This step takes a bit over a minute. 

In [ ]:
!mamba install -y lsdtopotools &> /dev/null

The next line tests to see if it worked. If you get some output asking for a parameter file then `lsdtopotools` is installed. 

In [ ]:
!lsdtt-basic-metrics

## First get data

Before we do anything, we need to import a few packages:

In [1]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
import lsdviztools.lsdmapwrappers as lsdmw

Now we need to get some data to download. We are going to download data using the opentopography scraper that is included with `lsdviztools`. You will need to get an opentopography.org account and copy in your API key.

You can sign up to an opentopography.org account here: https://portal.opentopography.org/myopentopo 

Before I actually do anything I am going to set up some filenames:

In [2]:
Dataset_prefix = "RioAguas"
source_name = "COP30"

r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"

DataDirectory = "./"
Base_file = r_prefix

Now lets grab the data. If you want to do this yourself for a new area just choose your own lower lect and upper right coordinates of your site

In [ ]:
# YOU NEED TO PUT YOUR API KEY IN A FILE
your_OT_api_key_file = "my_OT_api_key.txt"

with open(your_OT_api_key_file, 'r') as file:
    print("I am reading you OT API key from the file "+your_OT_api_key_file)
    api_key = file.read().rstrip()
    print("Your api key starts with: "+api_key[0:4])

SB_DEM = bmt.ot_scraper(source = source_name,
                        lower_left_coordinates = [36.97524478026287, -2.3631792251411805], 
                        upper_right_coordinates = [37.3200098350942, -1.7962073552766233],
                        prefix = Dataset_prefix, 
                        api_key_file = your_OT_api_key_file)
SB_DEM.print_parameters()
SB_DEM.download_pythonic()
DataDirectory = "./"
Fname = Dataset_prefix+"_"+source_name+".tif"
gio.convert4lsdtt(DataDirectory,Fname)

## Look at the hillshade

Right, lets see what this place looks like:

In [ ]:
lsdtt_parameters = {"write_hillshade" : "true"}
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
%matplotlib inline
Base_file = r_prefix
DataDirectory = "./"
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", save_fig=False, size_format="geomorphology",dpi=500)

## Now get a single basin

I add a basin outlet into a pandas dataframe and then copy this to a file. 
The points below are obtained just by clicking in google maps and copying the resulting lat-long into the below code. 

In [3]:
# Import pandas library
import pandas as pd

data = [ [37.15674383710805, -1.9049454817508027]]

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['latitude', 'longitude'])

df.to_csv("basin_outlets.csv",index=False)
df.head()

,latitude,longitude
0,37.156744,-1.904945


We can use the linux `cat` command to make sure the file is what we expect.

In [4]:
!cat basin_outlets.csv

latitude,longitude
37.15674383710805,-1.9049454817508027


## The different kinds of channel files

You can get channels in a number of ways from `lsdtopotools`.

The following are available in `lsdtt-basic-metrics`:
    
* `print_channels_to_csv:  true` This prints basic information about the channels to a csv file.
* `print_chi_data_maps: true` This includes the chi metric and drainage area
* `use_extended_channel_data: true` This adds some additional data columns to the above two csv files.

You can also control the extent of the channel network with `threshold_contributing_pixels`; a bigger number means a shorter channel network. 

Lets try these methods out. 

### `print_channels_to_csv`

First we use the `print_channels_to_csv`. This will produce a channel csv with *CN* in the filename:

In [6]:
## Get the basins and the channel profile
lsdtt_parameters = {"print_channels_to_csv" : "true",
                    "get_basins_from_outlets" : "true",
                    "basin_outlet_csv" : "basin_outlets.csv",
                    "threshold_contributing_pixels" : "2000"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

The lsdtopotools command line tools available are: 
['lsdtt-basic-metrics', 'lsdtt-channel-extraction', 'lsdtt-chi-mapping', 'lsdtt-cosmo-tool', 'lsdtt-hillslope-channel-coupling', 'lsdtt-valley-metrics']
Testing has been done against lsdtopotools v0.7
The command line tool is: lsdtt-chi-mapping
The driver name is: Test_01
The read path is: ./
The write path is: ./
The read prefix is: RioAguas_COP30_UTM
The write prefix is: RioAguas_COP30_UTM
The parameter dictionary is:
{'print_channels_to_csv': 'true', 'get_basins_from_outlets': 'true', 'basin_outlet_csv': 'basin_outlets.csv', 'threshold_contributing_pixels': '2000'}
Done writing the driver file
I've finised writing the driver file. Let me run LSDTT for you.
|| Welcome to the chi mapping tool!                    ||
|| This program has a number of options to make chi    ||
|| plots and to map out slopes in chi space.           ||
|| This program was developed by Simon M. Mudd         ||
|| Fiona J. Clubb,    Boris Gailleton           

We can look at the data using `pandas`

In [11]:
# Import pandas library
import pandas as pd

df = pd.read_csv("RioAguas_COP30_UTM_CN.csv")
df.head()

,latitude,longitude,Junction Index,Stream Order,NI,receiver_NI,receiver_JI
0,37.318696,-2.143100,0,1,12491,10816,1
1,37.318966,-2.143096,0,1,10816,9248,1
2,37.319237,-2.143093,0,1,9248,7814,1
3,37.319507,-2.143090,0,1,7814,6513,1
4,37.319777,-2.143087,0,1,6513,5331,1


So this file includes some flow routing information about the network. It has spatial coordinates (`latitude` and `longitude`) and then various information about the flow routing. Each junction is numbered (this includes sources). The `Junction Index` is the number of the junction upstream. There is also the `receiver_JI` which is the downstream junction to which the channel flows. In addition there are `NI` and `reciever_NI` columns. The `NI` is the node index, which is a number assigned to the pixel (in computational terms it is the index into the flattened array that holds the data) and there is the node index of the receiver pixel (`reciever_NI`). There is also the Strahler stream order (`Stream Order`) of the pixel. 

**This does not include drainge area because this routine does not check if the basin is complete.**

This file is generally only used for plotting the location of the channel. In practise, we really only use the alternative function `get_chi_data_maps`. But we will show you what that does in a second. 

First though, lets look at what happends when you extend the data:

In [12]:
## Get the basins and the channel profile
lsdtt_parameters = {"print_channels_to_csv" : "true",
                    "get_basins_from_outlets" : "true",
                    "basin_outlet_csv" : "basin_outlets.csv",
                    "threshold_contributing_pixels" : "2000",
                    "use_extended_channel_data" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

The lsdtopotools command line tools available are: 
['lsdtt-basic-metrics', 'lsdtt-channel-extraction', 'lsdtt-chi-mapping', 'lsdtt-cosmo-tool', 'lsdtt-hillslope-channel-coupling', 'lsdtt-valley-metrics']
Testing has been done against lsdtopotools v0.7
The command line tool is: lsdtt-chi-mapping
The driver name is: Test_01
The read path is: ./
The write path is: ./
The read prefix is: RioAguas_COP30_UTM
The write prefix is: RioAguas_COP30_UTM
The parameter dictionary is:
{'print_channels_to_csv': 'true', 'get_basins_from_outlets': 'true', 'basin_outlet_csv': 'basin_outlets.csv', 'threshold_contributing_pixels': '2000', 'use_extended_channel_data': 'true'}
Done writing the driver file
I've finised writing the driver file. Let me run LSDTT for you.
|| Welcome to the chi mapping tool!                    ||
|| This program has a number of options to make chi    ||
|| plots and to map out slopes in chi space.           ||
|| This program was developed by Simon M. Mudd         ||
|| Fiona J.

In [13]:
!ls *.csv

RioAguas_COP30_UTM_AllBasinsInfo.csv  RioAguas_COP30_UTM_chi_data_map.csv
RioAguas_COP30_UTM_CN.csv	      Spain_2023_Grid_References.csv
RioAguas_COP30_UTM_SAbinned.csv       basin_outlets.csv
RioAguas_COP30_UTM_SAvertical.csv     updated_spain_site_infomation.csv


In [14]:
df = pd.read_csv("RioAguas_COP30_UTM_CN.csv")
df.head()

,latitude,longitude,Junction Index,Stream Order,NI,receiver_NI,elevation(m),receiver_JI,upstream_JI
0,37.318696,-2.143100,0,1,12491,10816,363.53165,1,0
1,37.318966,-2.143096,0,1,10816,9248,362.32285,1,0
2,37.319237,-2.143093,0,1,9248,7814,360.95370,1,0
3,37.319507,-2.143090,0,1,7814,6513,359.81876,1,0
4,37.319777,-2.143087,0,1,6513,5331,359.19662,1,0


So all this realy does is add the elevation to the same file. 

### Now for `get_chi_data_maps`

**Firstly, you need to know what the chi coordinate is. If you know this already, skip ahead to the next cell.**

To understand it, we start with Morisawa's law that says slope ($S$) is related to drianage area ($A$) via two empirical constants, the concavity index ($\theta$) and the steepness index ($k_s$):

$S = k_s A^{-\theta}$

$S$ is the same as $dz/dx$, the derivative of elevation. $x$ is the flow distance. If we integrate this equation we get:
    
$z(x) = z(x_b) + \Big(\frac{k_s}{{A_0}^{\theta}}\Big) \int_{x_b}^{x} \Big(\frac{A_0}{A(x)}\Big)^{\theta} dx$

where $x_b$ is some arbitrary base level, and $A_0$ is a reference drainage area (this is to ensure the integrand is dimensionless). We almost always set $A_0$ to 1 $m^2$. The integrand seems annoying and messy, but it actually fairly easy to calculate from topographic data (you are just adding drainage area along the length of the channel. You will never need to do this yourself, there is software for calculating the integrand. It also has dimensions length. So we can define a coordinate, $\chi$:

$\chi = \int_{x_b}^{x} \Big(\frac{A_0}{A(x)}\Big)^{\theta} dx$

which is just that integrand, but it looks nicer in the equation:

$z(x) = z(x_b) + \Big(\frac{k_s}{{A_0}^{\theta}}\Big) \chi$

Now, have a look at that last equation. This is the equation of a line! 

From your school maths you might remember a line being written as $y = mx+b$. In this case the gradient of the line is $\Big(\frac{k_s}{{A_0}^{\theta}}\Big)$.

If $A_0$ = 1 $m^2$, then the gradient of the line is the channel steepness index ($k_s$)! This happens to be a very convinient way of extracting the channel steepness, and you can do it with much better accuracy than with slope--area plots. 

If you want to read more this technique, which is not widely used in geomorphology, you can read this paper:

Perron, J.T., Royden, L., 2013. An integral approach to bedrock river profile analysis. Earth Surface Processes and Landforms 38, 570–576. https://doi.org/10.1002/esp.3302


Note: $\chi$ is the greek letter chi, which a Greek person would pronounce a bit like the English "he". But because most geomorphologsts did not benefit from a classical education they pronounce this letter "kai".

#### Why `get_chi_data_maps` is more restrictive than `print_channels_to_csv`

The `get_chi_data_maps` routine is more restrictive then the `print_channels_to_csv` because it enforces complete basins. The reason why is that you must calculate the chi coordinate using the drainage area, and if the drainage area is wrong then the chi coordinate is wrong. 

**A very common error in `lsdtopotools` is searching for channels in an incomplete basin. The program will reject the basin and you will not extract any channels.** 

Now, lets see what the `print_chi_data_maps` does:

In [17]:
## Get the basins and the channel profile
lsdtt_parameters = {"print_chi_data_maps" : "true",
                    "get_basins_from_outlets" : "true",
                    "basin_outlet_csv" : "basin_outlets.csv",
                    "threshold_contributing_pixels" : "2000"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

The lsdtopotools command line tools available are: 
['lsdtt-basic-metrics', 'lsdtt-channel-extraction', 'lsdtt-chi-mapping', 'lsdtt-cosmo-tool', 'lsdtt-hillslope-channel-coupling', 'lsdtt-valley-metrics']
Testing has been done against lsdtopotools v0.7
The command line tool is: lsdtt-chi-mapping
The driver name is: Test_01
The read path is: ./
The write path is: ./
The read prefix is: RioAguas_COP30_UTM
The write prefix is: RioAguas_COP30_UTM
The parameter dictionary is:
{'print_chi_data_maps': 'true', 'get_basins_from_outlets': 'true', 'basin_outlet_csv': 'basin_outlets.csv', 'threshold_contributing_pixels': '2000'}
Done writing the driver file
I've finised writing the driver file. Let me run LSDTT for you.
|| Welcome to the chi mapping tool!                    ||
|| This program has a number of options to make chi    ||
|| plots and to map out slopes in chi space.           ||
|| This program was developed by Simon M. Mudd         ||
|| Fiona J. Clubb,    Boris Gailleton             

This prints a file with `_chi_data_map.csv` in the filename. Lets load it:

In [19]:
df = pd.read_csv("RioAguas_COP30_UTM_chi_data_map.csv")
df.head()

,latitude,longitude,chi,elevation,flow_distance,drainage_area,source_key,basin_key
0,37.200617,-2.278642,8.4261,875.72,67078.0,1822500.0,0,0
1,37.200345,-2.278307,8.3947,871.90,67036.0,1845000.0,0,0
2,37.200072,-2.277972,8.3635,868.45,66993.0,1856700.0,0,0
3,37.199800,-2.277636,8.3323,865.21,66951.0,1867500.0,0,0
4,37.199529,-2.277639,8.3104,862.98,66921.0,1872900.0,0,0


So, what is all this? 

* `latitude` : the latitude in decimal degrees
* `longitude` : the longitude in decimal degrees   
* `chi` : the chi coordinate in metres
* `elevation` : the elevation in the same units as the DEM (usually metres)
* `flow_distance` : the flow distance **from the outlet**.
* `drainage_area` : the drainage area of the pixel in m$^2$
* `source_key` : each source pixel (or channel head) gets an integer key. Channels are coded by their sources. Longer channels overwrite shorter channels so the main stem channel (here defined as the longest channel) will have all pixels with the same `source_key`. 
* `basin_key` : Each basin is given a number denoted by the `basin_key`. The longest channel in each basin has the smallest source key in that basin. 

Now lets look at the extended version of this csv:

In [20]:
## Get the basins and the channel profile
lsdtt_parameters = {"print_chi_data_maps" : "true",
                    "get_basins_from_outlets" : "true",
                    "basin_outlet_csv" : "basin_outlets.csv",
                    "threshold_contributing_pixels" : "2000",
                    "use_extended_channel_data" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

The lsdtopotools command line tools available are: 
['lsdtt-basic-metrics', 'lsdtt-channel-extraction', 'lsdtt-chi-mapping', 'lsdtt-cosmo-tool', 'lsdtt-hillslope-channel-coupling', 'lsdtt-valley-metrics']
Testing has been done against lsdtopotools v0.7
The command line tool is: lsdtt-chi-mapping
The driver name is: Test_01
The read path is: ./
The write path is: ./
The read prefix is: RioAguas_COP30_UTM
The write prefix is: RioAguas_COP30_UTM
The parameter dictionary is:
{'print_chi_data_maps': 'true', 'get_basins_from_outlets': 'true', 'basin_outlet_csv': 'basin_outlets.csv', 'threshold_contributing_pixels': '2000', 'use_extended_channel_data': 'true'}
Done writing the driver file
I've finised writing the driver file. Let me run LSDTT for you.
|| Welcome to the chi mapping tool!                    ||
|| This program has a number of options to make chi    ||
|| plots and to map out slopes in chi space.           ||
|| This program was developed by Simon M. Mudd         ||
|| Fiona J. C

In [21]:
!ls *.csv

RioAguas_COP30_UTM_AllBasinsInfo.csv  RioAguas_COP30_UTM_chi_data_map.csv
RioAguas_COP30_UTM_CN.csv	      Spain_2023_Grid_References.csv
RioAguas_COP30_UTM_SAbinned.csv       basin_outlets.csv
RioAguas_COP30_UTM_SAvertical.csv     updated_spain_site_infomation.csv


In [22]:
df = pd.read_csv("RioAguas_COP30_UTM_chi_data_map.csv")
df.head()

,latitude,longitude,NI,receiver_NI,chi,elevation,flow_distance,drainage_area,source_key,basin_key,stream_order,receiver_junction,upstream_junction
0,37.200617,-2.278642,746074,747704,8.4261,875.72,67078.0,1822500.0,0,0,1,249,248
1,37.200345,-2.278307,747704,749334,8.3947,871.90,67036.0,1845000.0,0,0,1,249,248
2,37.200072,-2.277972,749334,750964,8.3635,868.45,66993.0,1856700.0,0,0,1,249,248
3,37.199800,-2.277636,750964,752593,8.3323,865.21,66951.0,1867500.0,0,0,1,249,248
4,37.199529,-2.277639,752593,754223,8.3104,862.98,66921.0,1872900.0,0,0,1,249,248


The extended data has many more data elements. These are repeated from the channel_data csv. But perhaps most useful is the `stream_order` data entry. 

## What about channel steepness?

If you want to be more complicated, you can use `lsdtt-chi-mapping` to extract the channel steepness. 

This uses the segmentation algorithm from Mudd et al 2014 (JGR-ES, https://doi.org/10.1002/2013JF002981).

To turn it on you need `"print_segmented_M_chi_map_to_csv" : "true"`

Here we set the concavity index (m_over_n) to `0.45`. You actually need to set this for the `get_chi_data_maps` as well but the default is 0.45. 

**Warning: this is computationally intensive and might take a while.**

In [23]:
lsdtt_parameters = {"m_over_n" : "0.45",
                    "print_segmented_M_chi_map_to_csv" : "true",
                    "get_basins_from_outlets" : "true",
                    "basin_outlet_csv" : "basin_outlets.csv"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

The lsdtopotools command line tools available are: 
['lsdtt-basic-metrics', 'lsdtt-channel-extraction', 'lsdtt-chi-mapping', 'lsdtt-cosmo-tool', 'lsdtt-hillslope-channel-coupling', 'lsdtt-valley-metrics']
Testing has been done against lsdtopotools v0.7
The command line tool is: lsdtt-chi-mapping
The driver name is: Test_01
The read path is: ./
The write path is: ./
The read prefix is: RioAguas_COP30_UTM
The write prefix is: RioAguas_COP30_UTM
The parameter dictionary is:
{'m_over_n': '0.45', 'print_segmented_M_chi_map_to_csv': 'true', 'get_basins_from_outlets': 'true', 'basin_outlet_csv': 'basin_outlets.csv'}
Done writing the driver file
I've finised writing the driver file. Let me run LSDTT for you.
|| Welcome to the chi mapping tool!                    ||
|| This program has a number of options to make chi    ||
|| plots and to map out slopes in chi space.           ||
|| This program was developed by Simon M. Mudd         ||
|| Fiona J. Clubb,    Boris Gailleton                  ||
